In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from splinter import Browser
import splinter
import numpy as np

In [2]:
# create variables to store scraped info
movie_titles = []
opening_amts = []
total_gross = []
per_of_total = []
num_of_theaters = []
open_date = []

In [3]:
# loop through pages to get data from all 1000 movies 

pages = np.arange(0, 801, 200)

for page in pages:
    page = requests.get('https://www.boxofficemojo.com/chart/top_opening_weekend/?offset=' + str(page))
    
    # Create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(page.text, 'lxml')
    
    titles = soup.find_all('td', class_ = 'a-text-left mojo-field-type-release mojo-cell-wide')
    for movie in titles:
        movie_titles.append(movie.select('a')[0].string)

    opening = soup.find_all('td', class_ = 'a-text-right mojo-field-type-money')
    for amt in opening:
        opening_amts.append(amt.string)

    total = soup.find_all('td', class_ = 'a-text-right mojo-field-type-money mojo-estimatable')
    for money in total:
        total_gross.append(money.string)

    percent_of_total = soup.find_all('td', class_ = 'a-text-right mojo-field-type-percent')
    for percent in percent_of_total:
        per_of_total.append(percent.string)

    theaters = soup.find_all('td', class_ = 'a-text-right mojo-field-type-positive_integer mojo-estimatable')
    for thr in theaters:
        num_of_theaters.append(thr.string)

    date = soup.find_all('td', class_ = 'a-text-left mojo-field-type-date a-nowrap')
    for day in date:
        open_date.append(day.string)

In [4]:
gross_df = pd.DataFrame({"Gross & Average": total_gross})

gross_df

,Gross & Average
0,"$858,373,000"
1,"$76,601"
2,"$678,815,482"
3,"$57,599"
4,"$936,662,225"
...,...
1995,"$8,405"
1996,"$60,240,295"
1997,"$7,454"
1998,"$54,870,175"


In [5]:
# strip punctuation and turn into integer
gross_df['Gross & Average'] = gross_df['Gross & Average'].map(lambda x: x.lstrip('$')).str.replace(',', '').astype(int)


# separate total gross from avg. per theater
avg_thr = gross_df[gross_df['Gross & Average'] < 100000].dropna().reset_index()['Gross & Average']
total_grss = gross_df[gross_df['Gross & Average'] > 100000].dropna().reset_index()['Gross & Average']



gross_df['Total Gross'] = total_grss
gross_df['Average per Theater'] = avg_thr

gross_df = gross_df.drop(columns=['Gross & Average']).dropna()
gross_df['Movie Title'] = movie_titles

gross_df

,Total Gross,Average per Theater,Movie Title
0,858373000.0,76601.0,Avengers: Endgame
1,678815482.0,57599.0,Avengers: Infinity War
2,936662225.0,59982.0,Star Wars: Episode VII - The Force Awakens
3,620181382.0,51987.0,Star Wars: Episode VIII - The Last Jedi
4,652270625.0,48855.0,Jurassic World
...,...,...,...
995,50856010.0,6954.0,Ouija
996,48071303.0,6469.0,Kick-Ass
997,42670410.0,8405.0,The Unborn
998,60240295.0,7454.0,The Cable Guy


In [6]:
# create DataFrame
opening_df = pd.DataFrame({"Movie Title": movie_titles, "Opening": opening_amts, "Total Gross": total_grss, "% of Total": per_of_total, "Theaters": num_of_theaters, "Average per Theater": avg_thr, "Date": open_date})
#pd.options.display.float_format = '{:,}'.format

#df['new_column_name'] = df['column_name'].map('{:,.2f}'.format)

#opening_df['Total Gross'] = '$' + (opening_df['Total Gross'].astype(float)).map('{:,.2f}'.format).astype(str)
#opening_df['Average per Theater'] = '$' + (opening_df['Average per Theater'].astype(float)).map('{:,.2f}'.format).astype(str)

opening_df


,Movie Title,Opening,Total Gross,% of Total,Theaters,Average per Theater,Date
0,Avengers: Endgame,"$357,115,007",858373000,41.6%,"4,662",76601,"Apr 26, 2019"
1,Avengers: Infinity War,"$257,698,183",678815482,38%,"4,474",57599,"Apr 27, 2018"
2,Star Wars: Episode VII - The Force Awakens,"$247,966,675",936662225,26.5%,"4,134",59982,"Dec 18, 2015"
3,Star Wars: Episode VIII - The Last Jedi,"$220,009,584",620181382,35.5%,"4,232",51987,"Dec 15, 2017"
4,Jurassic World,"$208,806,270",652270625,32%,"4,274",48855,"Jun 12, 2015"
...,...,...,...,...,...,...,...
995,Ouija,"$19,875,995",50856010,39.1%,"2,858",6954,"Oct 24, 2014"
996,Kick-Ass,"$19,828,687",48071303,41.2%,"3,065",6469,"Apr 16, 2010"
997,The Unborn,"$19,810,585",42670410,46.4%,"2,357",8405,"Jan 9, 2009"
998,The Cable Guy,"$19,806,226",60240295,32.9%,"2,657",7454,"Jun 14, 1996"


In [7]:
# strip punctuation and turn into integer
opening_df['Opening'] = opening_df['Opening'].map(lambda x: x.lstrip('$')).str.replace(',', '').astype(int)
opening_df['Theaters'] = opening_df['Theaters'].str.replace(',', '').astype(int)

opening_df

,Movie Title,Opening,Total Gross,% of Total,Theaters,Average per Theater,Date
0,Avengers: Endgame,357115007,858373000,41.6%,4662,76601,"Apr 26, 2019"
1,Avengers: Infinity War,257698183,678815482,38%,4474,57599,"Apr 27, 2018"
2,Star Wars: Episode VII - The Force Awakens,247966675,936662225,26.5%,4134,59982,"Dec 18, 2015"
3,Star Wars: Episode VIII - The Last Jedi,220009584,620181382,35.5%,4232,51987,"Dec 15, 2017"
4,Jurassic World,208806270,652270625,32%,4274,48855,"Jun 12, 2015"
...,...,...,...,...,...,...,...
995,Ouija,19875995,50856010,39.1%,2858,6954,"Oct 24, 2014"
996,Kick-Ass,19828687,48071303,41.2%,3065,6469,"Apr 16, 2010"
997,The Unborn,19810585,42670410,46.4%,2357,8405,"Jan 9, 2009"
998,The Cable Guy,19806226,60240295,32.9%,2657,7454,"Jun 14, 1996"


In [8]:
import datetime as dt 
opening_df['Date'] = pd.to_datetime(opening_df['Date'])
opening_df

,Movie Title,Opening,Total Gross,% of Total,Theaters,Average per Theater,Date
0,Avengers: Endgame,357115007,858373000,41.6%,4662,76601,2019-04-26
1,Avengers: Infinity War,257698183,678815482,38%,4474,57599,2018-04-27
2,Star Wars: Episode VII - The Force Awakens,247966675,936662225,26.5%,4134,59982,2015-12-18
3,Star Wars: Episode VIII - The Last Jedi,220009584,620181382,35.5%,4232,51987,2017-12-15
4,Jurassic World,208806270,652270625,32%,4274,48855,2015-06-12
...,...,...,...,...,...,...,...
995,Ouija,19875995,50856010,39.1%,2858,6954,2014-10-24
996,Kick-Ass,19828687,48071303,41.2%,3065,6469,2010-04-16
997,The Unborn,19810585,42670410,46.4%,2357,8405,2009-01-09
998,The Cable Guy,19806226,60240295,32.9%,2657,7454,1996-06-14


In [9]:
#month = opening_df['Date'].dt.month
month = opening_df['Date']

opening_df['Season Number'] = (month.dt.month%12 + 3)//3
opening_df

,Movie Title,Opening,Total Gross,% of Total,Theaters,Average per Theater,Date,Season Number
0,Avengers: Endgame,357115007,858373000,41.6%,4662,76601,2019-04-26,2
1,Avengers: Infinity War,257698183,678815482,38%,4474,57599,2018-04-27,2
2,Star Wars: Episode VII - The Force Awakens,247966675,936662225,26.5%,4134,59982,2015-12-18,1
3,Star Wars: Episode VIII - The Last Jedi,220009584,620181382,35.5%,4232,51987,2017-12-15,1
4,Jurassic World,208806270,652270625,32%,4274,48855,2015-06-12,3
...,...,...,...,...,...,...,...,...
995,Ouija,19875995,50856010,39.1%,2858,6954,2014-10-24,4
996,Kick-Ass,19828687,48071303,41.2%,3065,6469,2010-04-16,2
997,The Unborn,19810585,42670410,46.4%,2357,8405,2009-01-09,1
998,The Cable Guy,19806226,60240295,32.9%,2657,7454,1996-06-14,3


In [10]:
year = opening_df['Date'].dt.year
opening_df['Release Year'] = year
opening_df

,Movie Title,Opening,Total Gross,% of Total,Theaters,Average per Theater,Date,Season Number,Release Year
0,Avengers: Endgame,357115007,858373000,41.6%,4662,76601,2019-04-26,2,2019
1,Avengers: Infinity War,257698183,678815482,38%,4474,57599,2018-04-27,2,2018
2,Star Wars: Episode VII - The Force Awakens,247966675,936662225,26.5%,4134,59982,2015-12-18,1,2015
3,Star Wars: Episode VIII - The Last Jedi,220009584,620181382,35.5%,4232,51987,2017-12-15,1,2017
4,Jurassic World,208806270,652270625,32%,4274,48855,2015-06-12,3,2015
...,...,...,...,...,...,...,...,...,...
995,Ouija,19875995,50856010,39.1%,2858,6954,2014-10-24,4,2014
996,Kick-Ass,19828687,48071303,41.2%,3065,6469,2010-04-16,2,2010
997,The Unborn,19810585,42670410,46.4%,2357,8405,2009-01-09,1,2009
998,The Cable Guy,19806226,60240295,32.9%,2657,7454,1996-06-14,3,1996


In [11]:
opening_df["Movie & Year"] = opening_df["Movie Title"] + " " + "(" + opening_df["Release Year"].astype(str) + ")"
opening_df.head()

,Movie Title,Opening,Total Gross,% of Total,Theaters,Average per Theater,Date,Season Number,Release Year,Movie & Year
0,Avengers: Endgame,357115007,858373000,41.6%,4662,76601,2019-04-26,2,2019,Avengers: Endgame (2019)
1,Avengers: Infinity War,257698183,678815482,38%,4474,57599,2018-04-27,2,2018,Avengers: Infinity War (2018)
2,Star Wars: Episode VII - The Force Awakens,247966675,936662225,26.5%,4134,59982,2015-12-18,1,2015,Star Wars: Episode VII - The Force Awakens (2015)
3,Star Wars: Episode VIII - The Last Jedi,220009584,620181382,35.5%,4232,51987,2017-12-15,1,2017,Star Wars: Episode VIII - The Last Jedi (2017)
4,Jurassic World,208806270,652270625,32%,4274,48855,2015-06-12,3,2015,Jurassic World (2015)


In [12]:
seasons = opening_df['Season Number'].astype(int)

season_name = []

for s in seasons:
    if (s == 1):
        season_name.append('Winter')
    elif (s == 2):
        season_name.append('Spring')
    elif (s == 3):
        season_name.append('Summer')
    elif (s == 4):
        season_name.append('Fall')

season_name

['Spring',
 'Spring',
 'Winter',
 'Winter',
 'Summer',
 'Spring',
 'Winter',
 'Summer',
 'Spring',
 'Summer',
 'Spring',
 'Winter',
 'Spring',
 'Spring',
 'Summer',
 'Spring',
 'Summer',
 'Summer',
 'Fall',
 'Winter',
 'Spring',
 'Spring',
 'Spring',
 'Summer',
 'Spring',
 'Spring',
 'Fall',
 'Fall',
 'Fall',
 'Summer',
 'Summer',
 'Summer',
 'Winter',
 'Fall',
 'Spring',
 'Spring',
 'Fall',
 'Fall',
 'Fall',
 'Fall',
 'Spring',
 'Summer',
 'Summer',
 'Summer',
 'Spring',
 'Summer',
 'Spring',
 'Spring',
 'Summer',
 'Summer',
 'Spring',
 'Spring',
 'Summer',
 'Spring',
 'Summer',
 'Spring',
 'Fall',
 'Fall',
 'Spring',
 'Summer',
 'Spring',
 'Spring',
 'Summer',
 'Spring',
 'Fall',
 'Spring',
 'Summer',
 'Fall',
 'Summer',
 'Spring',
 'Summer',
 'Spring',
 'Spring',
 'Spring',
 'Fall',
 'Spring',
 'Summer',
 'Fall',
 'Spring',
 'Spring',
 'Fall',
 'Fall',
 'Summer',
 'Spring',
 'Spring',
 'Fall',
 'Spring',
 'Winter',
 'Fall',
 'Spring',
 'Winter',
 'Spring',
 'Winter',
 'Summer',
 'Su

In [13]:
opening_df['Season Name'] = season_name

opening_df

,Movie Title,Opening,Total Gross,% of Total,Theaters,Average per Theater,Date,Season Number,Release Year,Movie & Year,Season Name
0,Avengers: Endgame,357115007,858373000,41.6%,4662,76601,2019-04-26,2,2019,Avengers: Endgame (2019),Spring
1,Avengers: Infinity War,257698183,678815482,38%,4474,57599,2018-04-27,2,2018,Avengers: Infinity War (2018),Spring
2,Star Wars: Episode VII - The Force Awakens,247966675,936662225,26.5%,4134,59982,2015-12-18,1,2015,Star Wars: Episode VII - The Force Awakens (2015),Winter
3,Star Wars: Episode VIII - The Last Jedi,220009584,620181382,35.5%,4232,51987,2017-12-15,1,2017,Star Wars: Episode VIII - The Last Jedi (2017),Winter
4,Jurassic World,208806270,652270625,32%,4274,48855,2015-06-12,3,2015,Jurassic World (2015),Summer
...,...,...,...,...,...,...,...,...,...,...,...
995,Ouija,19875995,50856010,39.1%,2858,6954,2014-10-24,4,2014,Ouija (2014),Fall
996,Kick-Ass,19828687,48071303,41.2%,3065,6469,2010-04-16,2,2010,Kick-Ass (2010),Spring
997,The Unborn,19810585,42670410,46.4%,2357,8405,2009-01-09,1,2009,The Unborn (2009),Winter
998,The Cable Guy,19806226,60240295,32.9%,2657,7454,1996-06-14,3,1996,The Cable Guy (1996),Summer


In [14]:
# export to csv
opening_df.to_csv('data/opening.csv')